In [11]:
import cv2, numpy as np

In [6]:
def extract_motion(delayed_frame, current_frame):
    result_frame = cv2.absdiff(current_frame, delayed_frame)
    result_frame = cv2.bitwise_not(result_frame)

    return result_frame

In [7]:
def adjust_saturation(image, saturation_scale=1.5):
    # Convert the image from BGR to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    
    # Scale the saturation channel
    s = np.clip(s * saturation_scale, 0, 255).astype(np.uint8)
    
    # Merge channels and convert back to BGR
    adjusted_hsv = cv2.merge((h, s, v))
    adjusted_image = cv2.cvtColor(adjusted_hsv, cv2.COLOR_HSV2BGR)

    return adjusted_image

In [102]:
def enhance_edges(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray_image, threshold1=100, threshold2=150, apertureSize=3)
    
    # Optional: Overlay edges on original image
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    enhanced_image = cv2.addWeighted(image, 0.8, edges_colored, 0.2, 0)
    return edges_colored

In [104]:
frames = []
path = "../app_test_data/Untitled.mp4"
cap = cv2.VideoCapture(path)
_, d_frame = cap.read()
count = 1

while True:
    # Capture frame-by-frames
    ret, c_frame = cap.read()

    if not ret:
        print('end')
        break

    c_frame = adjust_saturation(c_frame, 2.5)
    c_frame = enhance_edges(c_frame)
    out_frame = extract_motion(d_frame, c_frame)

    # Show the frame with the annotations
    cv2.imshow('asd', out_frame)
    d_frame = c_frame

    # Press 'q' to stop
    if cv2.waitKey(1) == ord('q'):
        break
    
    """
    zeros = ""
    for i in range(4-len(str(count))):
        zeros += "0"
    cv2.imwrite("images/frame_" + zeros + str(count) + ".png", out_frame)
    count += 1"""

cap.release()
cv2.destroyAllWindows()